In [1]:
import requests
import pandas as pd
import geopandas as gpd
import json
import numpy as np

### Part 1
Scraping School map data from MNPS site (https://mnps.maps.arcgis.com/home/item.html?id=a25c1ef058a247cfa2636a396f3aaedd#data). Table has information for geographic location (latitude and longitude) for each MNPS school. This information will be used for map generation. 

In [2]:
URL = 'https://services.arcgis.com/GBzOxI4K7WHYqcYy/arcgis/rest/services/Schools_Map/FeatureServer/0/query?f=json&where=1%3D1&returnGeometry=false&spatialRel=esriSpatialRelIntersects&outFields=*&orderByFields=OBJECTID%20ASC&resultOffset=0&resultRecordCount=1000&cacheHint=true&quantizationParameters=%7B%22mode%22%3A%22edit%22%7D'

In [3]:
response = requests.get(URL)

In [4]:
school_map_data = pd.DataFrame([x['attributes'] for x in response.json()['features']])

In [5]:
#school_map_data = pd.read_csv('Data/School_map_data.csv')
#school_map_data

In [6]:
school_map_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171 entries, 0 to 170
Data columns (total 28 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   OBJECTID        171 non-null    int64  
 1   CODE            171 non-null    int64  
 2   SHORT_NAME      171 non-null    object 
 3   SCHOOL_NAME     171 non-null    object 
 4   CLUSTER         171 non-null    int64  
 5   CLUS_NAME       171 non-null    object 
 6   GR_TIER         171 non-null    object 
 7   GRADES          171 non-null    object 
 8   TYPE_1          171 non-null    object 
 9   ZONE_1          171 non-null    object 
 10  NOTES           171 non-null    object 
 11  WEB_LINK        170 non-null    object 
 12  SHORT_ADDR      171 non-null    object 
 13  CITY            171 non-null    object 
 14  ZIP             171 non-null    object 
 15  PHONE           170 non-null    object 
 16  METRO_DIST      171 non-null    int64  
 17  SCHBRD_DIST     171 non-null    int

In [7]:
school_map_data = school_map_data.drop(columns = {'OBJECTID', 'SHORT_NAME', 'GR_TIER', 'NOTES', 'SCHBRD_MBR', 'OPTIONSCH_CURR', 'OPTIONSCH_NEXT', 'OPTID_CURR', 'OPTID_NEXT', 'TN_HOUSE', 'TN_SENATE'})
school_map_data = school_map_data.rename(columns = {'CLUS_NAME':'CLUSTER_NAME','LONG_':'LONGITUDE', 'LAT_':'LATITUDE'})
school_map_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171 entries, 0 to 170
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   CODE          171 non-null    int64  
 1   SCHOOL_NAME   171 non-null    object 
 2   CLUSTER       171 non-null    int64  
 3   CLUSTER_NAME  171 non-null    object 
 4   GRADES        171 non-null    object 
 5   TYPE_1        171 non-null    object 
 6   ZONE_1        171 non-null    object 
 7   WEB_LINK      170 non-null    object 
 8   SHORT_ADDR    171 non-null    object 
 9   CITY          171 non-null    object 
 10  ZIP           171 non-null    object 
 11  PHONE         170 non-null    object 
 12  METRO_DIST    171 non-null    int64  
 13  SCHBRD_DIST   171 non-null    int64  
 14  LONGITUDE     169 non-null    float64
 15  LATITUDE      169 non-null    float64
 16  CLUSTER_GEO   170 non-null    object 
dtypes: float64(2), int64(4), object(11)
memory usage: 22.8+ KB


In [8]:
school_map_data["SCHOOL_NAME"].sort_values().unique()

array(['A. Z. Kelley Elementary School', 'Alex Green Elementary School',
       'Amqui Elementary School', 'Andrew Jackson Elementary School',
       'Antioch High School', 'Antioch Middle School',
       'Apollo Middle School', 'Aventura Community School',
       'Bellshire Elementary School', 'Bellvue Middle School',
       'Board of Education', 'Brick Church: A LEAD Public School',
       'Cambridge Early Learning Center', 'Cameron College Preparatory',
       'Cane Ridge Elementary School', 'Cane Ridge High School',
       'Carlisle Beasley, Jr Building (Transportation)',
       'Carter-Lawrence Elementary School',
       'Casa Azafran Early Learning Center', 'Chadwell Elementary School',
       'Charlotte Park Elementary School', 'Cockrill Elementary School',
       'Cole Elementary School', 'Cora Howe School',
       'Crieve Hall Elemenary School', 'Croft Middle School',
       'Cumberland Elementary School', 'Dan Mills Elementary School',
       'Dodson Elementary School', 'Done

In [9]:
school_map_data['SCHOOL_NAME'] = school_map_data['SCHOOL_NAME'].replace({'Crieve Hall Elemenary School': 'Crieve Hall Elementary School',
                                                                         'DuPont Elementary School': 'Dupont Elementary School',
                                                                         'DuPont Hadley Middle School' :'Dupont-Hadley Middle School',
                                                                         'DuPont Tyler Middle School' : 'Dupont-Tyler Middle School',
                                                                         'East Nashville Magnet Middle School' : 'East Nashville Middle School',
                                                                         'East Nashville Magnet High School': 'East Nashville School',
                                                                         'H. G. Hill Middle School':'H.G. Hill Middle School',
                                                                         'Henry C. Maxwell Elementary School':'Henry C Maxwell Elementary School',
                                                                        'Intrepid College Preparatory Charter School':'Intrepid College Prep Charter School',
                                                                        'Ivanetta H. Davis Early Learning Center at Bordeaux': 'Ivanetta H. Davis Learning Center at Bordeaux School', 
                                                                        'J. E. Moss Elementary School':'J.E. Moss Elementary School',
                                                                        'KA @ The Crossings':'KA @ The Crossings School',
                                                                        'LEAD Academy High':'LEAD Academy School',
                                                                        'LEAD Prep Southeast':  'LEAD Southeast School',
                                                                       'Martin Luther King Jr. School' : 'Martin Luther King Jr School',
                                                                       'May Werthan Shayne Elemenatary School': 'May Werthan Shayne Elementary School',
                                                                        'MNPS Virtual School' : 'Metro Nashville Virtual School',
                                                                        'Murrell at Glenn School' :'Murrell School',
                                                                        'Nashville Big Picture High School':'Nashville Diploma Plus High School', 
                                                                         "Neely's Bend Elementary School" : 'Neelys Bend Elementary School',
                                                                         "Neely's Bend: A LEAD Public School": 'Neelys Bend Middle School',
                                                                         'Rocketship United Academy': 'Rocketship United School',
                                                                         'Ross Early Learning Center': 'Ross Elementary School',
                                                                         'Stratford STEM Magnet School - Upper/Lower Campus' : 'Stratford STEM Magnet School'
                                                                    
                                                                        })

In [10]:
school_map_data['ADDRESS'] = school_map_data.SHORT_ADDR.str.cat(school_map_data.CITY, sep=', ')
school_map_data['ADDRESS'] = school_map_data['ADDRESS'].astype(str) + ', TN-'
school_map_data['ADDRESS'] = school_map_data.ADDRESS.str.cat(school_map_data.ZIP, sep='')
school_map_data

,CODE,SCHOOL_NAME,CLUSTER,CLUSTER_NAME,GRADES,TYPE_1,ZONE_1,WEB_LINK,SHORT_ADDR,CITY,ZIP,PHONE,METRO_DIST,SCHBRD_DIST,LONGITUDE,LATITUDE,CLUSTER_GEO,ADDRESS
0,120,Jere Baxter Middle School,550,MAPLEWOOD,6-8,DISTRICT,ZONE SCHOOL,https://jerebaxter.mnps.org/,350 Hart Ln,Nashville,37207,615-262-6710,5,1,-86.750621,36.218945,MAPLEWOOD,"350 Hart Ln, Nashville, TN-37207"
1,200,Chadwell Elementary School,550,MAPLEWOOD,PK-5,DISTRICT,ZONE SCHOOL,https://chadwell.mnps.org/,321 Port Dr,Madison,37115,615-860-1459,3,1,-86.741676,36.256377,MAPLEWOOD,"321 Port Dr, Madison, TN-37115"
2,710,Stratton Elementary School,452,HUNTERS LANE,PK-5,DISTRICT,ZONE SCHOOL,https://stratton.mnps.org/,310 Old Hickory Blvd W,Madison,37115,615-860-1486,3,3,-86.717687,36.265610,HUNTERS LANE,"310 Old Hickory Blvd W, Madison, TN-37115"
3,545,Madison Middle School,452,HUNTERS LANE,6-8,DISTRICT,ZONE SCHOOL,https://madisonmiddle.mnps.org/,300 Old Hickory Blvd,Madison,37115,615-687-4018,3,3,-86.716452,36.264548,HUNTERS LANE,"300 Old Hickory Blvd, Madison, TN-37115"
4,550,Maplewood High School,550,MAPLEWOOD,9-12,DISTRICT,ZONE SCHOOL,https://maplewood.mnps.org/,401 Walton Ln,Nashville,37216,615-262-6770,5,1,-86.750472,36.232819,MAPLEWOOD,"401 Walton Ln, Nashville, TN-37216"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,285,John Early Middle School,632,PEARL-COHN,6-8,MAGNET,ZONE SCHOOL,https://johnearly.mnps.org/,1000 Cass St,Nashville,37208,615-291-6369,2,5,-86.804970,36.187034,PEARL-COHN,"1000 Cass St, Nashville, TN-37208"
167,350,Goodlettsville Elementary School,452,HUNTERS LANE,PK-5,DISTRICT,ZONE SCHOOL,https://goodlettsvilleelementary.mnps.org/,575 Joyce Ln,Nashville,37216,615-859-8950,7,1,NaN,NaN,MAPLEWOOD,"575 Joyce Ln, Nashville, TN-37216"
168,179,Aventura Community School,325,GLENCLIFF,K-1,CHARTER,NOT ZONE SCHOOL,None,3010 Tuggle Ave,Nashville,37211,None,16,7,NaN,NaN,None,"3010 Tuggle Ave, Nashville, TN-37211"
169,9999,Goodlettsville Elementary School,452,HUNTERS LANE,PK-4,DISTRICT,ZONE SCHOOL,https://goodlettsvilleelementary.mnps.org/,514 Donald St,Goodlettsville,37072,615-859-8950,10,3,-86.710655,36.313092,HUNTERS LANE,"514 Donald St, Goodlettsville, TN-37072"


In [11]:
enrollment = pd.read_csv('Data/enrollment.csv')
enrollment

,school,sl,enrollment,years
0,A. Z. Kelley Elementary,Elementary School,621,07-08
1,Alex Green Elementary,Elementary School,286,07-08
2,Amqui Elementary,Elementary School,667,07-08
3,Andrew Jackson Elementary,Elementary School,556,07-08
4,Antioch High School,High School,2403,07-08
...,...,...,...,...
2508,West End Middle,Middle School,343,22-23
2509,Westmeade Elementary,Elementary School,373,22-23
2510,Whites Creek High School,High School,601,22-23
2511,William Henry Oliver Middle,Middle School,669,22-23


In [12]:
enrollment = enrollment.rename(columns = {'school': 'SCHOOL_NAME', 'sl':'SCHOOL_TYPE', 'enrollment' : 'TOTAL_ENROLLMENT', 'years' : 'YEARS'})
enrollment

,SCHOOL_NAME,SCHOOL_TYPE,TOTAL_ENROLLMENT,YEARS
0,A. Z. Kelley Elementary,Elementary School,621,07-08
1,Alex Green Elementary,Elementary School,286,07-08
2,Amqui Elementary,Elementary School,667,07-08
3,Andrew Jackson Elementary,Elementary School,556,07-08
4,Antioch High School,High School,2403,07-08
...,...,...,...,...
2508,West End Middle,Middle School,343,22-23
2509,Westmeade Elementary,Elementary School,373,22-23
2510,Whites Creek High School,High School,601,22-23
2511,William Henry Oliver Middle,Middle School,669,22-23


In [13]:
enrollment["SCHOOL_NAME"].unique()

array(['A. Z. Kelley Elementary', 'Alex Green Elementary',
       'Amqui Elementary', 'Andrew Jackson Elementary',
       'Antioch High School', 'Antioch Middle', 'Apollo Middle',
       'Bailey Middle', 'Bellevue Middle', 'Bellshire Elementary',
       'Bordeaux Enhanced Option School', 'Brick Church Middle School',
       'Brookmeade Elementary School', 'Buena Vista Elementary',
       'Cameron Middle School', 'Carter-Lawrence Elementary',
       'Chadwell Elementary', 'Charlotte Park Elementary',
       'Cockrill Elementary', 'Cohn Adult Learning Center',
       'Cohn Alternative Center', 'Cole Elementary', 'Cora Howe School',
       'Creswell Middle School of the Arts', 'Crieve Hall Elementary',
       'Croft Middle', 'Cumberland Elementary', 'Dalewood Middle School',
       'Dan Mills Elementary', 'Dodson Elementary', 'Donelson Middle',
       'Dupont Elementary', 'Dupont-Hadley Middle', 'Dupont-Tyler Middle',
       'Eakin Elementary', 'Early College High School',
       'East Na

In [14]:
enrollment["SCHOOL_NAME"].nunique()

200

In [15]:
enrollment['SCHOOL_NAME'] = enrollment['SCHOOL_NAME'].replace({'Antioch High School': 'Antioch High' ,
                                   'Whites Creek High School': 'Whites Creek High', 
                                   'Bordeaux Enhanced Option School' : 'Bordeaux Enhanced Option',
                                   'Brick Church Middle School' : 'Brick Church: A LEAD Public',
                                   'Brookmeade Elementary School' : 'Brookmeade Elementary',
                                   'Cameron Middle School' : 'Cameron Middle',
                                    'Cora Howe School' : 'Cora Howe', 
                                    'Dalewood Middle School' : 'Dalewood Middle', 
                                    'Early College High School' : 'Early College High', 
                                    'East Nashville School' : 'East Nashville', 
                                    'Ewing Park Middle School' : 'Ewing Park Middle', 
                                    'Glencliff High School' : 'Glencliff High',
                                    'H.G. Hill Middle School' : 'H.G. Hill Middle', 
                                    'Martha Vaught Middle School': 'Martha Vaught Middle', 
                                   'Martin Luther King Jr School': 'Martin Luther King Jr',
                                   'May Werthan Shayne Elementary School' : 'May Werthan Shayne Elementary',
                                     'Ross Elementary School' : 'Ross Elementary', 
                                    'Stratford STEM Magnet School': 'Stratford STEM Magnet', 
                                    'W.A. Bass Middle School' : 'W.A. Bass Middle', 
                                     'Cane Ridge High School' :  'Cane Ridge High', 
                                    'Murrell School' :  'Murrell', 
                                    'Nashville Diploma Plus High School' : 'Nashville Diploma Plus High', 
                                    'Smithson Craighead Middle School' : 'Smithson Craighead Middle', 
                                    'Transition School' : 'Transition', 
                                    'East End Preparatory School' : 'East End Preparatory',
                                   'Metro Nashville Virtual School' : 'Metro Nashville Virtual',
                                    'Robertson Academy Gifted School' : 'Robertson Academy Gifted',
                                    'The Cohn School' : 'The Cohn',
                                    'KIPP Nashville Collegiate High School': 'KIPP Nashville Collegiate High',
                                    'Explore Community School' : 'Explore Community',
                                    'Knowledge Academies High School' : 'Knowledge Academies High',
                                    'STEM Prep High School' : 'STEM Prep High', 
                                    'RePublic High School' : 'RePublic High', 
                                    'Smith Springs Elementary School' : 'Smith Springs Elementary',                            
                                    'Aventura Community School' : 'Aventura Community',
                                    'Nashville Big Picture High School' : 'Nashville Big Picture High',
                                    'Waverly-Belmont Elementary School' : 'Waverly-Belmont Elementary',
                                    'Johnson School' : 'Johnson',
                                     'Bellevue Middle' : 'Bellvue Middle',                        
                                       'Cameron Middle School' :'Cameron College Preparatory'
                                         
                          })

In [16]:
enrollment["SCHOOL_NAME"].sort_values().unique()

array(['A. Z. Kelley Elementary', 'Alex Green Elementary',
       'Amqui Elementary', 'Andrew Jackson Elementary', 'Antioch High',
       'Antioch Middle', 'Apollo Middle', 'Aventura Community',
       'Bailey Middle', 'Bellshire Elementary', 'Bellvue Middle',
       'Bordeaux Enhanced Option', 'Boys Prep',
       'Brick Church College Prep', 'Brick Church: A LEAD Public',
       'Brookmeade Elementary', 'Buena Vista Elementary',
       'Cambridge Early Learning Center', 'Cameron College Preparatory',
       'Cane Ridge Elementary', 'Cane Ridge High',
       'Carter-Lawrence Elementary', 'Casa Azafran Early Learning Center',
       'Chadwell Elementary', 'Charlotte Park Elementary',
       'Cockrill Elementary', 'Cohn Adult Learning Center',
       'Cohn Alternative Center', 'Cole Elementary', 'Cora Howe',
       'Creswell Middle School of the Arts', 'Crieve Hall Elementary',
       'Croft Middle', 'Cumberland Elementary', 'Dalewood Middle',
       'Dan Mills Elementary', 'Dodson Eleme

In [17]:
enrollment["SCHOOL_NAME"].nunique()

200

In [18]:
enrollment['SCHOOL_NAME'] = enrollment['SCHOOL_NAME'].astype(str) + ' School'
enrollment

,SCHOOL_NAME,SCHOOL_TYPE,TOTAL_ENROLLMENT,YEARS
0,A. Z. Kelley Elementary School,Elementary School,621,07-08
1,Alex Green Elementary School,Elementary School,286,07-08
2,Amqui Elementary School,Elementary School,667,07-08
3,Andrew Jackson Elementary School,Elementary School,556,07-08
4,Antioch High School,High School,2403,07-08
...,...,...,...,...
2508,West End Middle School,Middle School,343,22-23
2509,Westmeade Elementary School,Elementary School,373,22-23
2510,Whites Creek High School,High School,601,22-23
2511,William Henry Oliver Middle School,Middle School,669,22-23


In [19]:
enrollment["SCHOOL_NAME"].unique()

array(['A. Z. Kelley Elementary School', 'Alex Green Elementary School',
       'Amqui Elementary School', 'Andrew Jackson Elementary School',
       'Antioch High School', 'Antioch Middle School',
       'Apollo Middle School', 'Bailey Middle School',
       'Bellvue Middle School', 'Bellshire Elementary School',
       'Bordeaux Enhanced Option School',
       'Brick Church: A LEAD Public School',
       'Brookmeade Elementary School', 'Buena Vista Elementary School',
       'Cameron College Preparatory School',
       'Carter-Lawrence Elementary School', 'Chadwell Elementary School',
       'Charlotte Park Elementary School', 'Cockrill Elementary School',
       'Cohn Adult Learning Center School',
       'Cohn Alternative Center School', 'Cole Elementary School',
       'Cora Howe School', 'Creswell Middle School of the Arts School',
       'Crieve Hall Elementary School', 'Croft Middle School',
       'Cumberland Elementary School', 'Dalewood Middle School',
       'Dan Mills Elem

In [20]:
enrollment['SCHOOL_NAME'] = enrollment['SCHOOL_NAME'].replace({'Liberty Collegiate Academy School' :  'Liberty Collegiate Academy',   
'Ross Early Learning Center School': 'Ross Early Learning Center',
'Nashville Classical School': 'Nashville Classical',
'Purpose Prep School': 'Purpose Prep', 
'Rocketship Nashville Northeast Elementary School':'Rocketship Nashville Northeast Elementary',
'Ivanetta H. Davis Early Learning Center at Bordeaux School': 'Ivanetta H. Davis Early Learning Center at Bordeaux',
'Stratford STEM Magnet School - Upper/Lower Campus School': 'Stratford STEM Magnet School - Upper/Lower Campus',
'Smithson Craighead Academy School':'Smithson Craighead Academy',
'LEAD Prep Southeast School': 'LEAD Prep Southeast',
'Strive Collegiate Academy School':'Strive Collegiate Academy',
'The Academy at Opry Mills School':'The Academy at Opry Mills',
'Rocketship United Academy School':'Rocketship United Academy',
'Harris-Hillman Special Education School':'Harris-Hillman Special Education',
'Johnson Alternative Learning Center School':'Johnson Alternative Learning Center',
'Nashville School Of The Arts School' : 'Nashville School of the Arts',
'Cameron College Preparatory School':'Cameron College Preparatory',
'Casa Azafran Early Learning Center School' :'Casa Azafran Early Learning Center', 
'STEM Prep Academy School' :'STEM Prep Academy',
'LEAD Academy High School':'LEAD Academy High',
'The Academy at Old Cockrill School' :'The Academy at Old Cockrill',
'W. A. Bass Learning Center - Adult School' : 'W. A. Bass Learning Center - Adult',
'W. A. Bass Learning Center - Transitions School':'W. A. Bass Learning Center - Transitions',
'Nashville Prep School' : 'Nashville Prep', 
'Tusculum Elementary School' :'Tusculum Elementary', 
'Valor Flagship Academy School' :'Valor Flagship Academy',
'Valor Voyager Academy School' :'Valor Voyager Academy',
'John Trotwood Moore Middle School': 'John Trotwood Moore Middle',
'Robertson Academy School':  'Robertson Academy', 
'KIPP Academy Nashville School':'KIPP Academy Nashville',
'Martin Professional Center School' : 'Martin Professional Center',
'Board of Education School' :'Board of Education',
'The Academy at Hickory Hollow School' : 'The Academy at Hickory Hollow',
'Cambridge Early Learning Center School': 'Cambridge Early Learning Center',
'KA @ The Crossings MS School' :'KA @ The Crossings', 
'KIPP Nashville College Prep School' : 'KIPP Nashville College Prep',
'KIPP Nashville College Prep Elementary School':'KIPP Nashville College Prep Elementary',
'W. A. Bass Learning Center - ALC School' : 'W. A. Bass Learning Center - ALC',
'Carlisle Beasley, Jr Building (Transportation) School' : 'Carlisle Beasley, Jr Building (Transportation)',
'MNPS Supply Center School' : 'MNPS Supply Center',
'MNPS Spencer Lane Warehouse School' : 'MNPS Spencer Lane Warehouse',
'McGruder Family Resource Center School':'McGruder Family Resource Center', 
'MNPS Maint/Supply - GENESCO School' : 'MNPS Maint/Supply - GENESCO',
'Nashville Collegiate Prep School' : 'Nashville Collegiate Prep' ,
'Robertson Academy Gifted School': 'Robertson Academy',
 'Smithson Craighead Middle School':'Smithson Craighead Academy'                                                        
})

In [21]:
school_enrollment = pd.merge(left = school_map_data, 
         right = enrollment, 
         on = 'SCHOOL_NAME',
        how = 'inner'            
        )

In [22]:
school_enrollment

,CODE,SCHOOL_NAME,CLUSTER,CLUSTER_NAME,GRADES,TYPE_1,ZONE_1,WEB_LINK,SHORT_ADDR,CITY,...,PHONE,METRO_DIST,SCHBRD_DIST,LONGITUDE,LATITUDE,CLUSTER_GEO,ADDRESS,SCHOOL_TYPE,TOTAL_ENROLLMENT,YEARS
0,120,Jere Baxter Middle School,550,MAPLEWOOD,6-8,DISTRICT,ZONE SCHOOL,https://jerebaxter.mnps.org/,350 Hart Ln,Nashville,...,615-262-6710,5,1,-86.750621,36.218945,MAPLEWOOD,"350 Hart Ln, Nashville, TN-37207",Middle School,492,07-08
1,120,Jere Baxter Middle School,550,MAPLEWOOD,6-8,DISTRICT,ZONE SCHOOL,https://jerebaxter.mnps.org/,350 Hart Ln,Nashville,...,615-262-6710,5,1,-86.750621,36.218945,MAPLEWOOD,"350 Hart Ln, Nashville, TN-37207",Middle School,483,08-09
2,120,Jere Baxter Middle School,550,MAPLEWOOD,6-8,DISTRICT,ZONE SCHOOL,https://jerebaxter.mnps.org/,350 Hart Ln,Nashville,...,615-262-6710,5,1,-86.750621,36.218945,MAPLEWOOD,"350 Hart Ln, Nashville, TN-37207",Middle School,505,09-10
3,120,Jere Baxter Middle School,550,MAPLEWOOD,6-8,DISTRICT,ZONE SCHOOL,https://jerebaxter.mnps.org/,350 Hart Ln,Nashville,...,615-262-6710,5,1,-86.750621,36.218945,MAPLEWOOD,"350 Hart Ln, Nashville, TN-37207",Middle School,529,10-11
4,120,Jere Baxter Middle School,550,MAPLEWOOD,6-8,DISTRICT,ZONE SCHOOL,https://jerebaxter.mnps.org/,350 Hart Ln,Nashville,...,615-262-6710,5,1,-86.750621,36.218945,MAPLEWOOD,"350 Hart Ln, Nashville, TN-37207",Middle School,494,11-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2226,9999,Goodlettsville Elementary School,452,HUNTERS LANE,PK-4,DISTRICT,ZONE SCHOOL,https://goodlettsvilleelementary.mnps.org/,514 Donald St,Goodlettsville,...,615-859-8950,10,3,-86.710655,36.313092,HUNTERS LANE,"514 Donald St, Goodlettsville, TN-37072",Elementary School,358,19-20
2227,9999,Goodlettsville Elementary School,452,HUNTERS LANE,PK-4,DISTRICT,ZONE SCHOOL,https://goodlettsvilleelementary.mnps.org/,514 Donald St,Goodlettsville,...,615-859-8950,10,3,-86.710655,36.313092,HUNTERS LANE,"514 Donald St, Goodlettsville, TN-37072",Elementary School,323,20-21
2228,9999,Goodlettsville Elementary School,452,HUNTERS LANE,PK-4,DISTRICT,ZONE SCHOOL,https://goodlettsvilleelementary.mnps.org/,514 Donald St,Goodlettsville,...,615-859-8950,10,3,-86.710655,36.313092,HUNTERS LANE,"514 Donald St, Goodlettsville, TN-37072",Elementary School,329,21-22
2229,9999,Goodlettsville Elementary School,452,HUNTERS LANE,PK-4,DISTRICT,ZONE SCHOOL,https://goodlettsvilleelementary.mnps.org/,514 Donald St,Goodlettsville,...,615-859-8950,10,3,-86.710655,36.313092,HUNTERS LANE,"514 Donald St, Goodlettsville, TN-37072",Elementary School,397,22-23


In [23]:
school_enrollment["SCHOOL_NAME"]. nunique()

158

In [24]:
school_enrollment["SCHOOL_NAME"].sort_values().unique()

array(['A. Z. Kelley Elementary School', 'Alex Green Elementary School',
       'Amqui Elementary School', 'Andrew Jackson Elementary School',
       'Antioch High School', 'Antioch Middle School',
       'Apollo Middle School', 'Aventura Community School',
       'Bellshire Elementary School', 'Bellvue Middle School',
       'Brick Church: A LEAD Public School',
       'Cambridge Early Learning Center', 'Cameron College Preparatory',
       'Cane Ridge Elementary School', 'Cane Ridge High School',
       'Carter-Lawrence Elementary School',
       'Casa Azafran Early Learning Center', 'Chadwell Elementary School',
       'Charlotte Park Elementary School', 'Cockrill Elementary School',
       'Cole Elementary School', 'Cora Howe School',
       'Crieve Hall Elementary School', 'Croft Middle School',
       'Cumberland Elementary School', 'Dan Mills Elementary School',
       'Dodson Elementary School', 'Donelson Middle School',
       'Dupont Elementary School', 'Dupont-Hadley Middle 

In [25]:
school_enrollment['YEARS'] = school_enrollment['YEARS'].replace({'07-08': '2007-2008',
                                    '08-09': '2008-2009',
                                    '09-10': '2009-2010',
                                    '10-11': '2010-2011',
                                    '11-12': '2011-2012',
                                    '12-13': '2012-2013', 
                                    '13-14': '2013-2014',
                                    '14-15': '2014-2015',
                                    '15-16': '2015-2016',
                                    '16-17': '2016-2017', 
                                    '17-18': '2017-2018',
                                    '18-19': '2018-2019',
                                    '19-20': '2019-2020',
                                    '20-21': '2020-2021',
                                    '21-22': '2021-2022',
                                    '22-23': '2022-2023'})

In [26]:
school_enrollment

,CODE,SCHOOL_NAME,CLUSTER,CLUSTER_NAME,GRADES,TYPE_1,ZONE_1,WEB_LINK,SHORT_ADDR,CITY,...,PHONE,METRO_DIST,SCHBRD_DIST,LONGITUDE,LATITUDE,CLUSTER_GEO,ADDRESS,SCHOOL_TYPE,TOTAL_ENROLLMENT,YEARS
0,120,Jere Baxter Middle School,550,MAPLEWOOD,6-8,DISTRICT,ZONE SCHOOL,https://jerebaxter.mnps.org/,350 Hart Ln,Nashville,...,615-262-6710,5,1,-86.750621,36.218945,MAPLEWOOD,"350 Hart Ln, Nashville, TN-37207",Middle School,492,2007-2008
1,120,Jere Baxter Middle School,550,MAPLEWOOD,6-8,DISTRICT,ZONE SCHOOL,https://jerebaxter.mnps.org/,350 Hart Ln,Nashville,...,615-262-6710,5,1,-86.750621,36.218945,MAPLEWOOD,"350 Hart Ln, Nashville, TN-37207",Middle School,483,2008-2009
2,120,Jere Baxter Middle School,550,MAPLEWOOD,6-8,DISTRICT,ZONE SCHOOL,https://jerebaxter.mnps.org/,350 Hart Ln,Nashville,...,615-262-6710,5,1,-86.750621,36.218945,MAPLEWOOD,"350 Hart Ln, Nashville, TN-37207",Middle School,505,2009-2010
3,120,Jere Baxter Middle School,550,MAPLEWOOD,6-8,DISTRICT,ZONE SCHOOL,https://jerebaxter.mnps.org/,350 Hart Ln,Nashville,...,615-262-6710,5,1,-86.750621,36.218945,MAPLEWOOD,"350 Hart Ln, Nashville, TN-37207",Middle School,529,2010-2011
4,120,Jere Baxter Middle School,550,MAPLEWOOD,6-8,DISTRICT,ZONE SCHOOL,https://jerebaxter.mnps.org/,350 Hart Ln,Nashville,...,615-262-6710,5,1,-86.750621,36.218945,MAPLEWOOD,"350 Hart Ln, Nashville, TN-37207",Middle School,494,2011-2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2226,9999,Goodlettsville Elementary School,452,HUNTERS LANE,PK-4,DISTRICT,ZONE SCHOOL,https://goodlettsvilleelementary.mnps.org/,514 Donald St,Goodlettsville,...,615-859-8950,10,3,-86.710655,36.313092,HUNTERS LANE,"514 Donald St, Goodlettsville, TN-37072",Elementary School,358,2019-2020
2227,9999,Goodlettsville Elementary School,452,HUNTERS LANE,PK-4,DISTRICT,ZONE SCHOOL,https://goodlettsvilleelementary.mnps.org/,514 Donald St,Goodlettsville,...,615-859-8950,10,3,-86.710655,36.313092,HUNTERS LANE,"514 Donald St, Goodlettsville, TN-37072",Elementary School,323,2020-2021
2228,9999,Goodlettsville Elementary School,452,HUNTERS LANE,PK-4,DISTRICT,ZONE SCHOOL,https://goodlettsvilleelementary.mnps.org/,514 Donald St,Goodlettsville,...,615-859-8950,10,3,-86.710655,36.313092,HUNTERS LANE,"514 Donald St, Goodlettsville, TN-37072",Elementary School,329,2021-2022
2229,9999,Goodlettsville Elementary School,452,HUNTERS LANE,PK-4,DISTRICT,ZONE SCHOOL,https://goodlettsvilleelementary.mnps.org/,514 Donald St,Goodlettsville,...,615-859-8950,10,3,-86.710655,36.313092,HUNTERS LANE,"514 Donald St, Goodlettsville, TN-37072",Elementary School,397,2022-2023


In [27]:
school_enrollment['START_YEAR'] = school_enrollment['YEARS'].str.split('-', expand = True)[0]

In [28]:
school_enrollment['END_YEAR'] = school_enrollment['YEARS'].str.split('-', expand = True)[1]

In [29]:
school_enrollment

,CODE,SCHOOL_NAME,CLUSTER,CLUSTER_NAME,GRADES,TYPE_1,ZONE_1,WEB_LINK,SHORT_ADDR,CITY,...,SCHBRD_DIST,LONGITUDE,LATITUDE,CLUSTER_GEO,ADDRESS,SCHOOL_TYPE,TOTAL_ENROLLMENT,YEARS,START_YEAR,END_YEAR
0,120,Jere Baxter Middle School,550,MAPLEWOOD,6-8,DISTRICT,ZONE SCHOOL,https://jerebaxter.mnps.org/,350 Hart Ln,Nashville,...,1,-86.750621,36.218945,MAPLEWOOD,"350 Hart Ln, Nashville, TN-37207",Middle School,492,2007-2008,2007,2008
1,120,Jere Baxter Middle School,550,MAPLEWOOD,6-8,DISTRICT,ZONE SCHOOL,https://jerebaxter.mnps.org/,350 Hart Ln,Nashville,...,1,-86.750621,36.218945,MAPLEWOOD,"350 Hart Ln, Nashville, TN-37207",Middle School,483,2008-2009,2008,2009
2,120,Jere Baxter Middle School,550,MAPLEWOOD,6-8,DISTRICT,ZONE SCHOOL,https://jerebaxter.mnps.org/,350 Hart Ln,Nashville,...,1,-86.750621,36.218945,MAPLEWOOD,"350 Hart Ln, Nashville, TN-37207",Middle School,505,2009-2010,2009,2010
3,120,Jere Baxter Middle School,550,MAPLEWOOD,6-8,DISTRICT,ZONE SCHOOL,https://jerebaxter.mnps.org/,350 Hart Ln,Nashville,...,1,-86.750621,36.218945,MAPLEWOOD,"350 Hart Ln, Nashville, TN-37207",Middle School,529,2010-2011,2010,2011
4,120,Jere Baxter Middle School,550,MAPLEWOOD,6-8,DISTRICT,ZONE SCHOOL,https://jerebaxter.mnps.org/,350 Hart Ln,Nashville,...,1,-86.750621,36.218945,MAPLEWOOD,"350 Hart Ln, Nashville, TN-37207",Middle School,494,2011-2012,2011,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2226,9999,Goodlettsville Elementary School,452,HUNTERS LANE,PK-4,DISTRICT,ZONE SCHOOL,https://goodlettsvilleelementary.mnps.org/,514 Donald St,Goodlettsville,...,3,-86.710655,36.313092,HUNTERS LANE,"514 Donald St, Goodlettsville, TN-37072",Elementary School,358,2019-2020,2019,2020
2227,9999,Goodlettsville Elementary School,452,HUNTERS LANE,PK-4,DISTRICT,ZONE SCHOOL,https://goodlettsvilleelementary.mnps.org/,514 Donald St,Goodlettsville,...,3,-86.710655,36.313092,HUNTERS LANE,"514 Donald St, Goodlettsville, TN-37072",Elementary School,323,2020-2021,2020,2021
2228,9999,Goodlettsville Elementary School,452,HUNTERS LANE,PK-4,DISTRICT,ZONE SCHOOL,https://goodlettsvilleelementary.mnps.org/,514 Donald St,Goodlettsville,...,3,-86.710655,36.313092,HUNTERS LANE,"514 Donald St, Goodlettsville, TN-37072",Elementary School,329,2021-2022,2021,2022
2229,9999,Goodlettsville Elementary School,452,HUNTERS LANE,PK-4,DISTRICT,ZONE SCHOOL,https://goodlettsvilleelementary.mnps.org/,514 Donald St,Goodlettsville,...,3,-86.710655,36.313092,HUNTERS LANE,"514 Donald St, Goodlettsville, TN-37072",Elementary School,397,2022-2023,2022,2023


Exporting data frame into csv file for use in R 

In [30]:
school_enrollment.to_csv('school_enrollment.csv', index = True)

### Part 2

Preparing the data for School Demographics and Ethinicity. Data was scraped from the MNPS Dashboard and stored in csv files

In [31]:
A_Z_Kelley_Elementary = pd.read_csv('detailed_by_school/A. Z. Kelley Elementary.csv')
Alex_Green_Elementary = pd.read_csv('detailed_by_school/Alex Green Elementary.csv')
Amqui_Elementary = pd.read_csv('detailed_by_school/Amqui Elementary.csv')
Andrew_Jackson_Elementary = pd.read_csv('detailed_by_school/Andrew Jackson Elementary.csv')
Antioch_High_School = pd.read_csv('detailed_by_school/Antioch High School.csv')
Antioch_Middle = pd.read_csv('detailed_by_school/Antioch Middle.csv')
Apollo_Middle = pd.read_csv('detailed_by_school/Apollo Middle.csv')
Aventura_Community_School = pd.read_csv('detailed_by_school/Aventura Community School.csv')
#Bailey_Middle = pd.read_csv('detailed_by_school/ Bailey Middle.csv')
Bellevue_Middle = pd.read_csv('detailed_by_school/Bellevue Middle.csv')
Bellshire_Elementary = pd.read_csv('detailed_by_school/Bellshire Elementary.csv')
#Bordeaux_Enhanced_Option_School = pd.read_csv('detailed_by_school/Bordeaux Enhanced Option School.csv')
#Boys_Prep = pd.read_csv('detailed_by_school/Boys Prep.csv')
#Brick_Church_College_Prep = pd.read_csv('detailed_by_school/Brick Church College Prep.csv')
Brick_Church_Middle_School = pd.read_csv('detailed_by_school/Brick Church Middle School.csv')
#Brookmeade_Elementary_School = pd.read_csv('detailed_by_school/Brookmeade Elementary School.csv')
#Buena_Vista_Elementary = pd.read_csv('detailed_by_school/Buena Vista Elementary.csv')
Cambridge_Early_Learning_Center = pd.read_csv('detailed_by_school/Cambridge Early Learning Center.csv')
Cameron_Middle_School= pd.read_csv('detailed_by_school/Cameron Middle School.csv')
Cane_Ridge_Elementary = pd.read_csv('detailed_by_school/Cane Ridge Elementary.csv')
Cane_Ridge_High_School = pd.read_csv('detailed_by_school/Cane Ridge High School.csv')
Carter_Lawrence_Elementary = pd.read_csv('detailed_by_school/Carter-Lawrence Elementary.csv')
Casa_Azafran_Early_Learning_Center = pd.read_csv('detailed_by_school/Casa Azafran Early Learning Center.csv')
Chadwell_Elementary = pd.read_csv('detailed_by_school/Chadwell Elementary.csv')
Charlotte_Park_Elementary = pd.read_csv('detailed_by_school/Charlotte Park Elementary.csv')
Cockrill_Elementary = pd.read_csv('detailed_by_school/Cockrill Elementary.csv')
#Cohn Adult Learning Center = pd.read_csv('detailed_by_school/Cohn Adult Learning Center.csv')
#Cohn Alternative Center = pd.read_csv('detailed_by_school/Cohn Alternative Center.csv')
Cole_Elementary = pd.read_csv('detailed_by_school/Cole Elementary.csv')
Cora_Howe_School = pd.read_csv('detailed_by_school/Cora Howe School.csv')
#Creswell Middle School of the Arts = pd.read_csv('detailed_by_school/Creswell Middle School of the Arts.csv')
Crieve_Hall_Elementary = pd.read_csv('detailed_by_school/Crieve Hall Elementary.csv')
Croft_Middle = pd.read_csv('detailed_by_school/Croft Middle.csv')
Cumberland_Elementary = pd.read_csv('detailed_by_school/Cumberland Elementary.csv')
#Dalewood Middle School = pd.read_csv('detailed_by_school/Dalewood Middle School.csv')
Dan_Mills_Elementary = pd.read_csv('detailed_by_school/Dan Mills Elementary.csv')
Dodson_Elementary = pd.read_csv('detailed_by_school/Dodson Elementary.csv')
Donelson_Middle = pd.read_csv('detailed_by_school/Donelson Middle.csv')
#Drexel Prep = pd.read_csv('detailed_by_school/Drexel Prep.csv')
Dupont_Elementary = pd.read_csv('detailed_by_school/Dupont Elementary.csv')
Dupont_Hadley_Middle = pd.read_csv('detailed_by_school/Dupont-Hadley Middle.csv')
Dupont_Tyler_Middle = pd.read_csv('detailed_by_school/Dupont-Tyler Middle.csv')
Eagle_View_Elementary = pd.read_csv('detailed_by_school/Eagle View Elementary.csv')
Eakin_Elementary = pd.read_csv('detailed_by_school/Eakin Elementary.csv')
Early_College_High_School = pd.read_csv('detailed_by_school/Early College High School.csv')
East_End_Preparatory_School = pd.read_csv('detailed_by_school/East End Preparatory School.csv')
East_Nashville_Middle = pd.read_csv('detailed_by_school/East Nashville Middle.csv')
East_Nashville_School = pd.read_csv('detailed_by_school/East Nashville School.csv')
#Ewing Park Middle School = pd.read_csv('detailed_by_school/Ewing Park Middle School.csv')
Explore_Community_School = pd.read_csv('detailed_by_school/Explore Community School.csv')
Fall_Hamilton_Elementary = pd.read_csv('detailed_by_school/Fall-Hamilton Elementary.csv')
Gateway_Elementary = pd.read_csv('detailed_by_school/Gateway Elementary.csv')
Glencliff_Elementary = pd.read_csv('detailed_by_school/Glencliff Elementary.csv')
Glencliff_High_School = pd.read_csv('detailed_by_school/Glencliff High School.csv')
Glendale_Elementary = pd.read_csv('detailed_by_school/Glendale Elementary.csv')
Glengarry_Elementary = pd.read_csv('detailed_by_school/Glengarry Elementary.csv')
#Glenn Elementary = pd.read_csv('detailed_by_school/Glenn Elementary.csv')
Glenview_Elementary = pd.read_csv('detailed_by_school/Glenview Elementary.csv')
Goodlettsville_Elementary = pd.read_csv('detailed_by_school/Goodlettsville Elementary.csv')
Goodlettsville_Middle = pd.read_csv('detailed_by_school/Goodlettsville Middle.csv')
Gower_Elementary = pd.read_csv('detailed_by_school/Gower Elementary.csv')
#Gra Mar Middle = pd.read_csv('detailed_by_school/Gra-Mar Middle.csv')
Granbery_Elementary = pd.read_csv('detailed_by_school/Granbery Elementary.csv')
H_G_Hill_Middle_School = pd.read_csv('detailed_by_school/H.G. Hill Middle School.csv')
Harpeth_Valley_Elementary = pd.read_csv('detailed_by_school/Harpeth Valley Elementary.csv')
Harris_Hillman_Special_Education = pd.read_csv('detailed_by_school/Harris-Hillman Special Education.csv')
Hattie_Cotton_Elementary = pd.read_csv('detailed_by_school/Hattie Cotton Elementary.csv')
Haynes_Middle = pd.read_csv('detailed_by_school/Haynes Middle.csv')
Haywood_Elementary = pd.read_csv('detailed_by_school/Haywood Elementary.csv')
Head_Middle = pd.read_csv('detailed_by_school/Head Middle.csv')
Henry_C_Maxwell_Elementary = pd.read_csv('detailed_by_school/Henry C Maxwell Elementary.csv')
Hermitage_Elementary = pd.read_csv('detailed_by_school/Hermitage Elementary.csv')
Hickman_Elementary = pd.read_csv('detailed_by_school/Hickman Elementary.csv')
Hillsboro_High = pd.read_csv('detailed_by_school/Hillsboro High.csv')
Hillwood_High = pd.read_csv('detailed_by_school/Hillwood High.csv')
Hull_Jackson_Elementary = pd.read_csv('detailed_by_school/Hull-Jackson Elementary.csv')
Hume_Fogg_High = pd.read_csv('detailed_by_school/Hume-Fogg High.csv')
Hunters_Lane_High = pd.read_csv('detailed_by_school/Hunters Lane High.csv')
Ida_B_Wells_Elementary = pd.read_csv('detailed_by_school/Ida B. Wells Elementary.csv')
Inglewood_Elementary = pd.read_csv('detailed_by_school/Inglewood Elementary.csv')
Intrepid_College_Prep_Charter = pd.read_csv('detailed_by_school/Intrepid College Prep Charter.csv')
Isaac_Litton_Middle = pd.read_csv('detailed_by_school/Isaac Litton Middle.csv')
Ivanetta_H_Davis_Learning_Center_at_Bordeaux = pd.read_csv('detailed_by_school/Ivanetta H. Davis Learning Center at Bordeaux.csv')
J_E_Moss_Elementary = pd.read_csv('detailed_by_school/J.E. Moss Elementary.csv')
#Jere Baxter Alternative Center = pd.read_csv('detailed_by_school/Jere Baxter Alternative Center.csv')
Jere_Baxter_Middle = pd.read_csv('detailed_by_school/Jere Baxter Middle.csv')
Joelton_Elementary = pd.read_csv('detailed_by_school/Joelton Elementary.csv')
#Joelton_Middle = pd.read_csv('detailed_by_school/Joelton Middle.csv')
John_B_Whitsitt_Elementary = pd.read_csv('detailed_by_school/John B. Whitsitt Elementary.csv')
John_Early_Middle = pd.read_csv('detailed_by_school/John Early Middle.csv')
John_F_Kennedy_Middle = pd.read_csv('detailed_by_school/John F. Kennedy Middle.csv')
John_Overton_High = pd.read_csv('detailed_by_school/John Overton High.csv')
John_Trotwood_Moore_Middle = pd.read_csv('detailed_by_school/John Trotwood Moore Middle.csv')
Johnson_Alternative_Learning_Center = pd.read_csv('detailed_by_school/Johnson Alternative Learning Center.csv')
#Johnson School = pd.read_csv('detailed_by_school/Johnson School.csv')
Jones_Elementary = pd.read_csv('detailed_by_school/Jones Elementary.csv')
Julia_Green_Elementary = pd.read_csv('detailed_by_school/Julia Green Elementary.csv')
#KA @ The Crossings = pd.read_csv('detailed_by_school/KA @ The Crossings.csv')
KA_The_Crossings_MS = pd.read_csv('detailed_by_school/KA @ The Crossings MS.csv')
KIPP_Academy_Nashville = pd.read_csv('detailed_by_school/KIPP Academy Nashville.csv')
KIPP_Academy_Nashville_Elementary = pd.read_csv('detailed_by_school/KIPP Academy Nashville Elementary.csv')
KIPP_Nashville_College_Prep = pd.read_csv('detailed_by_school/KIPP Nashville College Prep.csv')
KIPP_Nashville_College_Prep_Elementary = pd.read_csv('detailed_by_school/KIPP Nashville College Prep Elementary.csv')
KIPP_Nashville_Collegiate_High_School = pd.read_csv('detailed_by_school/KIPP Nashville Collegiate High School.csv')
#Kirkpatrick Elementary = pd.read_csv('detailed_by_school/Kirkpatrick Elementary.csv')
Knowledge_Academies_High_School = pd.read_csv('detailed_by_school/Knowledge Academies High School.csv')
#Knowledge Academy = pd.read_csv('detailed_by_school/Knowledge Academy.csv')
Lakeview_Elementary = pd.read_csv('detailed_by_school/Lakeview Elementary.csv')
LEAD_Academy = pd.read_csv('detailed_by_school/LEAD Academy.csv')
#LEAD Cameron = pd.read_csv('detailed_by_school/LEAD Cameron.csv')
LEAD_Southeast = pd.read_csv('detailed_by_school/LEAD Southeast.csv')
Liberty_Collegiate_Academy = pd.read_csv('detailed_by_school/Liberty Collegiate Academy.csv')
Lockeland_Elementary = pd.read_csv('detailed_by_school/Lockeland Elementary.csv')
Madison_Middle = pd.read_csv('detailed_by_school/Madison Middle.csv')
#Madison Special Education Center = pd.read_csv('detailed_by_school/Madison Special Education Center.csv')
Maplewood_High = pd.read_csv('detailed_by_school/Maplewood High.csv')
Margaret_Allen_Middle = pd.read_csv('detailed_by_school/Margaret Allen Middle.csv')
#Martha_Vaught_Middle_School = pd.read_csv('detailed_by_school/Martha Vaught Middle School.csv')
Martin_Luther_King_Jr_School = pd.read_csv('detailed_by_school/Martin Luther King Jr School.csv')
May_Werthan_Shayne_Elementary_School= pd.read_csv('detailed_by_school/May Werthan Shayne Elementary School.csv')
#McCann Alternative Learning Center = pd.read_csv('detailed_by_school/McCann Alternative Learning Center.csv')
McGavock_Elementary = pd.read_csv('detailed_by_school/McGavock Elementary.csv')
McGavock_High = pd.read_csv('detailed_by_school/McGavock High.csv')
#McKissack Professional Development = pd.read_csv('detailed_by_school/McKissack Professional Development.csv')
McMurray_Middle = pd.read_csv('detailed_by_school/McMurray Middle.csv')
Meigs_Middle = pd.read_csv('detailed_by_school/Meigs Middle.csv')
Metro_Nashville_Virtual_School = pd.read_csv('detailed_by_school/Metro Nashville Virtual School.csv')
#MNPS Middle School ALC = pd.read_csv('detailed_by_school/MNPS Middle School ALC.csv')
Moses_McKissack_Middle = pd.read_csv('detailed_by_school/Moses McKissack Middle.csv')
Mt_View_Elementary = pd.read_csv('detailed_by_school/Mt. View Elementary.csv')
Murrell_School = pd.read_csv('detailed_by_school/Murrell School.csv')
Napier_Elementary = pd.read_csv('detailed_by_school/Napier Elementary.csv')
#Nashville Academy of Computer Science = pd.read_csv('detailed_by_school/Nashville Academy of Computer Science.csv')
#Nashville_Big_Picture_High_School = pd.read_csv('detailed_by_school/Nashville Big Picture High School.csv')
Nashville_Classical = pd.read_csv('detailed_by_school/Nashville Classical.csv')
Nashville_Diploma_Plus_High_School = pd.read_csv('detailed_by_school/Nashville Diploma Plus High School.csv')
Nashville_Prep = pd.read_csv('detailed_by_school/Nashville Prep.csv')
Nashville_School_Of_The_Arts = pd.read_csv('detailed_by_school/Nashville School Of The Arts.csv')
#Neelys Bend College Prep = pd.read_csv('detailed_by_school/Neelys Bend College Prep.csv')
Neelys_Bend_Elementary = pd.read_csv('detailed_by_school/Neelys Bend Elementary.csv')
Neelys_Bend_Middle = pd.read_csv('detailed_by_school/Neelys Bend Middle.csv')
#New Vision Academy = pd.read_csv('detailed_by_school/New Vision Academy.csv')
Norman_Binkley_Elementary = pd.read_csv('detailed_by_school/Norman Binkley Elementary.csv')
Old_Center_Elementary = pd.read_csv('detailed_by_school/Old Center Elementary.csv')
Paragon_Mills_Elementary = pd.read_csv('detailed_by_school/Paragon Mills Elementary.csv')
Park_Avenue_Elementary = pd.read_csv('detailed_by_school/Park Avenue Elementary.csv')
Pearl_Cohn_High = pd.read_csv('detailed_by_school/Pearl-Cohn High.csv')
Pennington_Elementary= pd.read_csv('detailed_by_school/Pennington Elementary.csv')
Percy_Priest_Elementary = pd.read_csv('detailed_by_school/Percy Priest Elementary.csv')
Purpose_Prep = pd.read_csv('detailed_by_school/Purpose Prep.csv')
RePublic_High_School = pd.read_csv('detailed_by_school/RePublic High School.csv')
Robert_Churchwell_Elementary = pd.read_csv('detailed_by_school/Robert Churchwell Elementary.csv')
#Robert E. Lillard Elementary = pd.read_csv('detailed_by_school/Robert E. Lillard Elementary.csv')
Robertson_Academy_Gifted_School = pd.read_csv('detailed_by_school/Robertson Academy Gifted School.csv')
Rocketship_Nashville_Northeast_Elementary = pd.read_csv('detailed_by_school/Rocketship Nashville Northeast Elementary.csv')
#Rocketship Partners Community Prep = pd.read_csv('detailed_by_school/Rocketship Partners Community Prep.csv')
Rocketship_United = pd.read_csv('detailed_by_school/Rocketship United.csv')
Rose_Park_Middle = pd.read_csv('detailed_by_school/Rose Park Middle.csv')
Rosebank_Elementary = pd.read_csv('detailed_by_school/Rosebank Elementary.csv')
#Ross Early Learning Center = pd.read_csv('detailed_by_school/Ross Early Learning Center.csv')
Ross_Elementary_School = pd.read_csv('detailed_by_school/Ross Elementary School.csv')
Ruby_Major_Elementary = pd.read_csv('detailed_by_school/Ruby Major Elementary.csv')
Shwab_Elementary = pd.read_csv('detailed_by_school/Shwab Elementary.csv')
Smith_Springs_Elementary_School = pd.read_csv('detailed_by_school/Smith Springs Elementary School.csv')
Smithson_Craighead_Academy = pd.read_csv('detailed_by_school/Smithson Craighead Academy.csv')
#Smithson Craighead Middle School = pd.read_csv('detailed_by_school/Smithson Craighead Middle School.csv')
#Special Education Services = pd.read_csv('detailed_by_school/Special Education Services.csv')
Stanford_Elementary = pd.read_csv('detailed_by_school/Stanford Elementary.csv')
STEM_Prep_Academy = pd.read_csv('detailed_by_school/STEM Prep Academy.csv')
STEM_Prep_High_School = pd.read_csv('detailed_by_school/STEM Prep High School.csv')
Stratford_STEM_Magnet = School = pd.read_csv('detailed_by_school/Stratford STEM Magnet School.csv')
Stratton_Elementary = pd.read_csv('detailed_by_school/Stratton Elementary.csv')
Strive_Collegiate_Academy = pd.read_csv('detailed_by_school/Strive Collegiate Academy.csv')
Sylvan_Park_Elementary = pd.read_csv('detailed_by_school/Sylvan Park Elementary.csv')
The_Academy_at_Hickory_Hollow = pd.read_csv('detailed_by_school/The Academy at Hickory Hollow.csv')
The_Academy_at_Old_Cockrill = pd.read_csv('detailed_by_school/The Academy at Old Cockrill.csv')
The_Academy_at_Opry_Mills = pd.read_csv('detailed_by_school/The Academy at Opry Mills.csv')
#The Cohn Learning Center = pd.read_csv('detailed_by_school/The Cohn Learning Center.csv')
#The Cohn School = pd.read_csv('detailed_by_school/The Cohn School.csv')
Thomas_A_Edison_Elementary = pd.read_csv('detailed_by_school/Thomas A. Edison Elementary.csv')
Thurgood_Marshall_Middle = pd.read_csv('detailed_by_school/Thurgood Marshall Middle.csv')
Tom_Joy_Elementary = pd.read_csv('detailed_by_school/Tom Joy Elementary.csv')
#Transition School = pd.read_csv('detailed_by_school/Transition School.csv')
#Transitions at Bass = pd.read_csv('detailed_by_school/Transitions at Bass.csv')
Tulip_Grove_Elementary = pd.read_csv('detailed_by_school/Tulip Grove Elementary.csv')
Tusculum_Elementary = pd.read_csv('detailed_by_school/Tusculum Elementary.csv')
Two_Rivers_Middle = pd.read_csv('detailed_by_school/Two Rivers Middle.csv')
Una_Elementary = pd.read_csv('detailed_by_school/Una Elementary.csv')
Valor_Flagship_Academy = pd.read_csv('detailed_by_school/Valor Flagship Academy.csv')
Valor_Voyager_Academy = pd.read_csv('detailed_by_school/Valor Voyager Academy.csv')
#W. A. Bass Adult Program = pd.read_csv('detailed_by_school/W. A. Bass Adult Program.csv')
#W_A_Bass_Alternative_Learning_Center = pd.read_csv('detailed_by_school/W.A. Bass Alternative Learning Center.csv')
#W_A_Bass_Middle_School = pd.read_csv('detailed_by_school/W.A. Bass Middle School.csv')
Warner_Elementary = pd.read_csv('detailed_by_school/Warner Elementary.csv')
Waverly_Belmont_Elementary_School = pd.read_csv('detailed_by_school/Waverly-Belmont Elementary School.csv')
West_End_Middle = pd.read_csv('detailed_by_school/West End Middle.csv')
Westmeade_Elementary = pd.read_csv('detailed_by_school/Westmeade Elementary.csv')
Whites_Creek_High_School = pd.read_csv('detailed_by_school/Whites Creek High School.csv')
William_Henry_Oliver_Middle = pd.read_csv('detailed_by_school/William Henry Oliver Middle.csv') 
wright_middle = pd.read_csv('detailed_by_school/Wright Middle.csv')


In [32]:
Demographics = pd.concat((A_Z_Kelley_Elementary, Alex_Green_Elementary))
Demographics = pd.concat((Demographics, Amqui_Elementary))
Demographics = pd.concat((Demographics, Andrew_Jackson_Elementary))
Demographics = pd.concat((Demographics, Antioch_High_School))
Demographics = pd.concat((Demographics, Antioch_Middle))
Demographics = pd.concat((Demographics, Apollo_Middle))
Demographics = pd.concat((Demographics, Aventura_Community_School))
Demographics = pd.concat((Demographics, Bellevue_Middle))
Demographics = pd.concat((Demographics, Bellshire_Elementary))
Demographics = pd.concat((Demographics, Brick_Church_Middle_School))
Demographics = pd.concat((Demographics, Cambridge_Early_Learning_Center))
Demographics = pd.concat((Demographics, Cameron_Middle_School))
Demographics = pd.concat((Demographics, Cane_Ridge_Elementary))
Demographics = pd.concat((Demographics, Cane_Ridge_High_School))
Demographics = pd.concat((Demographics, Carter_Lawrence_Elementary))
Demographics = pd.concat((Demographics, Casa_Azafran_Early_Learning_Center))
Demographics = pd.concat((Demographics, Chadwell_Elementary))
Demographics = pd.concat((Demographics, Charlotte_Park_Elementary))
Demographics = pd.concat((Demographics, Cockrill_Elementary ))
Demographics = pd.concat((Demographics, Cole_Elementary))
Demographics = pd.concat((Demographics, Cora_Howe_School))
Demographics = pd.concat((Demographics, Crieve_Hall_Elementary))
Demographics = pd.concat((Demographics, Croft_Middle))
Demographics = pd.concat((Demographics, Cumberland_Elementary))
Demographics = pd.concat((Demographics, Dan_Mills_Elementary))
Demographics = pd.concat((Demographics, Dodson_Elementary))
Demographics = pd.concat((Demographics, Donelson_Middle))
Demographics = pd.concat((Demographics, Dupont_Elementary))
Demographics = pd.concat((Demographics, Dupont_Hadley_Middle))
Demographics = pd.concat((Demographics, Dupont_Tyler_Middle))
Demographics = pd.concat((Demographics, Eagle_View_Elementary))
Demographics = pd.concat((Demographics, Eakin_Elementary))
Demographics = pd.concat((Demographics, Early_College_High_School))
Demographics = pd.concat((Demographics, East_End_Preparatory_School))
Demographics = pd.concat((Demographics, East_Nashville_Middle ))
Demographics = pd.concat((Demographics, East_Nashville_School ))
Demographics = pd.concat((Demographics, Explore_Community_School))
Demographics = pd.concat((Demographics, Fall_Hamilton_Elementary))
Demographics = pd.concat((Demographics, Gateway_Elementary ))
Demographics = pd.concat((Demographics, Glencliff_Elementary))
Demographics = pd.concat((Demographics, Glencliff_High_School))
Demographics = pd.concat((Demographics, Glendale_Elementary ))
Demographics = pd.concat((Demographics, Glengarry_Elementary))
Demographics = pd.concat((Demographics, Glenview_Elementary))
Demographics = pd.concat((Demographics, Goodlettsville_Elementary))
Demographics = pd.concat((Demographics, Goodlettsville_Middle))
Demographics = pd.concat((Demographics, Gower_Elementary))
Demographics = pd.concat((Demographics, Granbery_Elementary))
Demographics = pd.concat((Demographics, H_G_Hill_Middle_School))
Demographics = pd.concat((Demographics, Harpeth_Valley_Elementary))
Demographics = pd.concat((Demographics, Harris_Hillman_Special_Education))
Demographics = pd.concat((Demographics, Hattie_Cotton_Elementary))
Demographics = pd.concat((Demographics, Haynes_Middle))
Demographics = pd.concat((Demographics, Haywood_Elementary))
Demographics = pd.concat((Demographics, Head_Middle))
Demographics = pd.concat((Demographics, Henry_C_Maxwell_Elementary))
Demographics = pd.concat((Demographics, Hermitage_Elementary))
Demographics = pd.concat((Demographics, Hickman_Elementary))
Demographics = pd.concat((Demographics, Hillsboro_High))
Demographics = pd.concat((Demographics, Hillwood_High))
Demographics = pd.concat((Demographics, Hull_Jackson_Elementary))
Demographics = pd.concat((Demographics, Hume_Fogg_High))
Demographics = pd.concat((Demographics, Hunters_Lane_High))
Demographics = pd.concat((Demographics, Ida_B_Wells_Elementary))
Demographics = pd.concat((Demographics, Inglewood_Elementary))
Demographics = pd.concat((Demographics, Intrepid_College_Prep_Charter ))
Demographics = pd.concat((Demographics, Isaac_Litton_Middle))
Demographics = pd.concat((Demographics, Ivanetta_H_Davis_Learning_Center_at_Bordeaux))
Demographics = pd.concat((Demographics, J_E_Moss_Elementary ))
Demographics = pd.concat((Demographics, Jere_Baxter_Middle ))
Demographics = pd.concat((Demographics, Joelton_Elementary))
#Demographics = pd.concat((Demographics, Joelton_Middle))
Demographics = pd.concat((Demographics, John_B_Whitsitt_Elementary ))
Demographics = pd.concat((Demographics, John_Early_Middle))
Demographics = pd.concat((Demographics, John_F_Kennedy_Middle ))
Demographics = pd.concat((Demographics, John_Overton_High))
Demographics = pd.concat((Demographics, John_Trotwood_Moore_Middle))
Demographics = pd.concat((Demographics, Johnson_Alternative_Learning_Center ))
Demographics = pd.concat((Demographics, Jones_Elementary))
Demographics = pd.concat((Demographics, Julia_Green_Elementary))
Demographics = pd.concat((Demographics, KA_The_Crossings_MS))
Demographics = pd.concat((Demographics, KIPP_Academy_Nashville))
Demographics = pd.concat((Demographics, KIPP_Academy_Nashville_Elementary))
Demographics = pd.concat((Demographics, KIPP_Nashville_College_Prep))
Demographics = pd.concat((Demographics, KIPP_Nashville_College_Prep_Elementary))
Demographics = pd.concat((Demographics, KIPP_Nashville_Collegiate_High_School))
Demographics = pd.concat((Demographics, Knowledge_Academies_High_School ))
Demographics = pd.concat((Demographics, Lakeview_Elementary))
Demographics = pd.concat((Demographics, LEAD_Academy))
Demographics = pd.concat((Demographics, LEAD_Southeast))
Demographics = pd.concat((Demographics, Liberty_Collegiate_Academy))
Demographics = pd.concat((Demographics, Lockeland_Elementary))
Demographics = pd.concat((Demographics, Madison_Middle))
Demographics = pd.concat((Demographics, Maplewood_High))
Demographics = pd.concat((Demographics, Margaret_Allen_Middle))
#Demographics = pd.concat((Demographics, Martha_Vaught_Middle_School))
Demographics = pd.concat((Demographics, Martin_Luther_King_Jr_School))
Demographics = pd.concat((Demographics, May_Werthan_Shayne_Elementary_School))
Demographics = pd.concat((Demographics, McGavock_Elementary))
Demographics = pd.concat((Demographics, McGavock_High))
Demographics = pd.concat((Demographics, McMurray_Middle))
Demographics = pd.concat((Demographics, Meigs_Middle))
Demographics = pd.concat((Demographics, Metro_Nashville_Virtual_School))
Demographics = pd.concat((Demographics, Moses_McKissack_Middle))
Demographics = pd.concat((Demographics, Mt_View_Elementary))
Demographics = pd.concat((Demographics, Murrell_School))
Demographics = pd.concat((Demographics, Napier_Elementary ))
#Demographics = pd.concat((Demographics, Nashville_Big_Picture_High_School))
Demographics = pd.concat((Demographics, Nashville_Classical))
Demographics = pd.concat((Demographics, Nashville_Diploma_Plus_High_School))
Demographics = pd.concat((Demographics, Nashville_Prep))
Demographics = pd.concat((Demographics, Nashville_School_Of_The_Arts))
Demographics = pd.concat((Demographics, Neelys_Bend_Elementary))
Demographics = pd.concat((Demographics, Neelys_Bend_Middle))
Demographics = pd.concat((Demographics, Norman_Binkley_Elementary))
Demographics = pd.concat((Demographics, Old_Center_Elementary))
Demographics = pd.concat((Demographics, Paragon_Mills_Elementary))
Demographics = pd.concat((Demographics, Park_Avenue_Elementary))
Demographics = pd.concat((Demographics, Pearl_Cohn_High))
Demographics = pd.concat((Demographics, Pennington_Elementary))
Demographics = pd.concat((Demographics, Percy_Priest_Elementary))
Demographics = pd.concat((Demographics, Purpose_Prep))
Demographics = pd.concat((Demographics, RePublic_High_School))
Demographics = pd.concat((Demographics, Robert_Churchwell_Elementary))
Demographics = pd.concat((Demographics, Robertson_Academy_Gifted_School))
Demographics = pd.concat((Demographics, Rocketship_Nashville_Northeast_Elementary))
Demographics = pd.concat((Demographics, Rocketship_United))
Demographics = pd.concat((Demographics, Rose_Park_Middle))
Demographics = pd.concat((Demographics, Rosebank_Elementary))
Demographics = pd.concat((Demographics, Ross_Elementary_School ))
Demographics = pd.concat((Demographics, Ruby_Major_Elementary))
Demographics = pd.concat((Demographics, Shwab_Elementary))
Demographics = pd.concat((Demographics, Smith_Springs_Elementary_School))
Demographics = pd.concat((Demographics, Smithson_Craighead_Academy))
Demographics = pd.concat((Demographics, Stanford_Elementary))
Demographics = pd.concat((Demographics, STEM_Prep_Academy))
Demographics = pd.concat((Demographics, STEM_Prep_High_School ))
Demographics = pd.concat((Demographics, Stratford_STEM_Magnet))
Demographics = pd.concat((Demographics, Stratton_Elementary))
Demographics = pd.concat((Demographics, Strive_Collegiate_Academy))
Demographics = pd.concat((Demographics, Sylvan_Park_Elementary))
Demographics = pd.concat((Demographics, The_Academy_at_Hickory_Hollow))
Demographics = pd.concat((Demographics, The_Academy_at_Old_Cockrill))
Demographics = pd.concat((Demographics, The_Academy_at_Opry_Mills))
Demographics = pd.concat((Demographics, Thomas_A_Edison_Elementary))
Demographics = pd.concat((Demographics, Thurgood_Marshall_Middle))
Demographics = pd.concat((Demographics, Tom_Joy_Elementary))
Demographics = pd.concat((Demographics, Tulip_Grove_Elementary))
Demographics = pd.concat((Demographics, Tusculum_Elementary))
Demographics = pd.concat((Demographics, Two_Rivers_Middle))
Demographics = pd.concat((Demographics, Una_Elementary))
Demographics = pd.concat((Demographics, Valor_Flagship_Academy))
Demographics = pd.concat((Demographics, Valor_Voyager_Academy))
Demographics = pd.concat((Demographics, Warner_Elementary))
Demographics = pd.concat((Demographics, Waverly_Belmont_Elementary_School))
Demographics = pd.concat((Demographics, West_End_Middle))
Demographics = pd.concat((Demographics, Westmeade_Elementary))
Demographics = pd.concat((Demographics, Whites_Creek_High_School))
Demographics = pd.concat((Demographics, William_Henry_Oliver_Middle))
Demographics = pd.concat((Demographics, wright_middle))
Demographics

,school,grade,gender,year,White,Hispanic/Latino,Black or African American,Asian,American Indian or Alaska Native,Native Hawaiian or Other Pacific Islander,N/A
0,A. Z. Kelley Elementary,P3,Female,08-09,1.0,0.0,0.0,0.0,0.0,0.0,NaN
1,A. Z. Kelley Elementary,P3,Female,09-10,2.0,2.0,0.0,0.0,0.0,0.0,NaN
2,A. Z. Kelley Elementary,P3,Female,11-12,1.0,0.0,5.0,0.0,0.0,0.0,NaN
3,A. Z. Kelley Elementary,P3,Female,12-13,1.0,0.0,1.0,0.0,0.0,0.0,NaN
4,A. Z. Kelley Elementary,P3,Female,14-15,1.0,0.0,0.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
123,Wright Middle,8,Male,18-19,15.0,55.0,17.0,1.0,0.0,0.0,NaN
124,Wright Middle,8,Male,19-20,13.0,65.0,24.0,3.0,0.0,0.0,NaN
125,Wright Middle,8,Male,20-21,14.0,77.0,13.0,2.0,0.0,0.0,NaN
126,Wright Middle,8,Male,21-22,11.0,81.0,17.0,3.0,0.0,0.0,NaN


In [33]:
Demographics["school"].nunique()

158

In [34]:
Demographics["school"].unique()

array(['A. Z. Kelley Elementary', 'Alex Green Elementary',
       'Amqui Elementary', 'Andrew Jackson Elementary',
       'Antioch High School', 'Antioch Middle', 'Apollo Middle',
       'Aventura Community School', 'Bellevue Middle',
       'Bellshire Elementary', 'Brick Church Middle School',
       'Cambridge Early Learning Center', 'Cameron Middle School',
       'Cane Ridge Elementary', 'Cane Ridge High School',
       'Carter-Lawrence Elementary', 'Casa Azafran Early Learning Center',
       'Chadwell Elementary', 'Charlotte Park Elementary',
       'Cockrill Elementary', 'Cole Elementary', 'Cora Howe School',
       'Crieve Hall Elementary', 'Croft Middle', 'Cumberland Elementary',
       'Dan Mills Elementary', 'Dodson Elementary', 'Donelson Middle',
       'Dupont Elementary', 'Dupont-Hadley Middle', 'Dupont-Tyler Middle',
       'Eagle View Elementary', 'Eakin Elementary',
       'Early College High School', 'East End Preparatory School',
       'East Nashville Middle', 'East

In [35]:
Demographics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23860 entries, 0 to 127
Data columns (total 11 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   school                                     23860 non-null  object 
 1   grade                                      23855 non-null  object 
 2   gender                                     23860 non-null  object 
 3   year                                       23860 non-null  object 
 4   White                                      23860 non-null  float64
 5   Hispanic/Latino                            23860 non-null  float64
 6   Black or African American                  23860 non-null  float64
 7   Asian                                      23767 non-null  float64
 8   American Indian or Alaska Native           22945 non-null  float64
 9   Native Hawaiian or Other Pacific Islander  21714 non-null  float64
 10  N/A                     

In [36]:
Demographics = Demographics.rename(columns = {'school': 'School_Name', 
                                          'grade':'Grade', 
                                          'gender' : 'Gender',
                                          'year' : 'Year'
                                             })
Demographics = Demographics.drop(columns = 'N/A')                                     
Demographics

,School_Name,Grade,Gender,Year,White,Hispanic/Latino,Black or African American,Asian,American Indian or Alaska Native,Native Hawaiian or Other Pacific Islander
0,A. Z. Kelley Elementary,P3,Female,08-09,1.0,0.0,0.0,0.0,0.0,0.0
1,A. Z. Kelley Elementary,P3,Female,09-10,2.0,2.0,0.0,0.0,0.0,0.0
2,A. Z. Kelley Elementary,P3,Female,11-12,1.0,0.0,5.0,0.0,0.0,0.0
3,A. Z. Kelley Elementary,P3,Female,12-13,1.0,0.0,1.0,0.0,0.0,0.0
4,A. Z. Kelley Elementary,P3,Female,14-15,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
123,Wright Middle,8,Male,18-19,15.0,55.0,17.0,1.0,0.0,0.0
124,Wright Middle,8,Male,19-20,13.0,65.0,24.0,3.0,0.0,0.0
125,Wright Middle,8,Male,20-21,14.0,77.0,13.0,2.0,0.0,0.0
126,Wright Middle,8,Male,21-22,11.0,81.0,17.0,3.0,0.0,0.0


In [37]:
Demographics['Year'] = Demographics['Year'].replace({'07-08': '2007-2008',
                                    '08-09': '2008-2009',
                                    '09-10': '2009-2010',
                                    '10-11': '2010-2011',
                                    '11-12': '2011-2012',
                                    '12-13': '2012-2013', 
                                    '13-14': '2013-2014',
                                    '14-15': '2014-2015',
                                    '15-16': '2015-2016',
                                    '16-17': '2016-2017', 
                                    '17-18': '2017-2018',
                                    '18-19': '2018-2019',
                                    '19-20': '2019-2020',
                                    '20-21': '2020-2021',
                                    '21-22': '2021-2022',
                                    '22-23': '2022-2023'})

In [38]:
Demographics['School_Name'] = Demographics['School_Name'].replace({'Antioch High School': 'Antioch High' ,
                                   'Whites Creek High School': 'Whites Creek High', 
                                   'Bordeaux Enhanced Option School' : 'Bordeaux Enhanced Option',
                                   'Brick Church Middle School' : 'Brick Church: A LEAD Public',
                                   'Brookmeade Elementary School' : 'Brookmeade Elementary',
                                   'Cameron Middle School' : 'Cameron Middle',
                                    'Cora Howe School' : 'Cora Howe', 
                                    'Dalewood Middle School' : 'Dalewood Middle', 
                                    'Early College High School' : 'Early College High', 
                                    'East Nashville School' : 'East Nashville', 
                                    'Ewing Park Middle School' : 'Ewing Park Middle', 
                                    'Glencliff High School' : 'Glencliff High',
                                    'H.G. Hill Middle School' : 'H.G. Hill Middle', 
                                    'Martha Vaught Middle School': 'Martha Vaught Middle', 
                                   'Martin Luther King Jr School': 'Martin Luther King Jr',
                                   'May Werthan Shayne Elementary School' : 'May Werthan Shayne Elementary',
                                     'Ross Elementary School' : 'Ross Elementary', 
                                    'Stratford STEM Magnet School': 'Stratford STEM Magnet', 
                                    'W.A. Bass Middle School' : 'W.A. Bass Middle', 
                                     'Cane Ridge High School' :  'Cane Ridge High', 
                                    'Murrell School' :  'Murrell', 
                                    'Nashville Diploma Plus High School' : 'Nashville Diploma Plus High', 
                                    'Smithson Craighead Middle School' : 'Smithson Craighead Middle', 
                                    'Transition School' : 'Transition', 
                                    'East End Preparatory School' : 'East End Preparatory',
                                   'Metro Nashville Virtual School' : 'Metro Nashville Virtual',
                                    'Robertson Academy Gifted School' : 'Robertson Academy Gifted',
                                    'The Cohn School' : 'The Cohn',
                                    'KIPP Nashville Collegiate High School': 'KIPP Nashville Collegiate High',
                                    'Explore Community School' : 'Explore Community',
                                    'Knowledge Academies High School' : 'Knowledge Academies High',
                                    'STEM Prep High School' : 'STEM Prep High', 
                                    'RePublic High School' : 'RePublic High', 
                                    'Smith Springs Elementary School' : 'Smith Springs Elementary',                            
                                    'Aventura Community School' : 'Aventura Community',
                                    'Nashville Big Picture High School' : 'Nashville Big Picture High',
                                    'Waverly-Belmont Elementary School' : 'Waverly-Belmont Elementary',
                                    'Johnson School' : 'Johnson',
                                     'Bellevue Middle' : 'Bellvue Middle',                        
                                       'Cameron Middle School' :'Cameron College Preparatory'
                                         
                          })

In [39]:
Demographics['School_Name'] = Demographics['School_Name'].astype(str) + ' School'

In [40]:
Demographics['School_Name'] = Demographics['School_Name'].replace({'Liberty Collegiate Academy School' :  'Liberty Collegiate Academy',   
'Ross Early Learning Center School': 'Ross Early Learning Center',
'Nashville Classical School': 'Nashville Classical',
'Purpose Prep School': 'Purpose Prep', 
'Rocketship Nashville Northeast Elementary School':'Rocketship Nashville Northeast Elementary',
'Ivanetta H. Davis Early Learning Center at Bordeaux School': 'Ivanetta H. Davis Early Learning Center at Bordeaux',
'Stratford STEM Magnet School - Upper/Lower Campus School': 'Stratford STEM Magnet School - Upper/Lower Campus',
'Smithson Craighead Academy School':'Smithson Craighead Academy',
'LEAD Prep Southeast School': 'LEAD Prep Southeast',
'Strive Collegiate Academy School':'Strive Collegiate Academy',
'The Academy at Opry Mills School':'The Academy at Opry Mills',
'Rocketship United Academy School':'Rocketship United Academy',
'Harris-Hillman Special Education School':'Harris-Hillman Special Education',
'Johnson Alternative Learning Center School':'Johnson Alternative Learning Center',
'Nashville School Of The Arts School' : 'Nashville School of the Arts',
'Cameron College Preparatory School':'Cameron College Preparatory',
'Casa Azafran Early Learning Center School' :'Casa Azafran Early Learning Center', 
'STEM Prep Academy School' :'STEM Prep Academy',
'LEAD Academy High School':'LEAD Academy High',
'The Academy at Old Cockrill School' :'The Academy at Old Cockrill',
'W. A. Bass Learning Center - Adult School' : 'W. A. Bass Learning Center - Adult',
'W. A. Bass Learning Center - Transitions School':'W. A. Bass Learning Center - Transitions',
'Nashville Prep School' : 'Nashville Prep', 
'Tusculum Elementary School' :'Tusculum Elementary', 
'Valor Flagship Academy School' :'Valor Flagship Academy',
'Valor Voyager Academy School' :'Valor Voyager Academy',
'John Trotwood Moore Middle School': 'John Trotwood Moore Middle',
'Robertson Academy School':  'Robertson Academy', 
'KIPP Academy Nashville School':'KIPP Academy Nashville',
'Martin Professional Center School' : 'Martin Professional Center',
'Board of Education School' :'Board of Education',
'The Academy at Hickory Hollow School' : 'The Academy at Hickory Hollow',
'Cambridge Early Learning Center School': 'Cambridge Early Learning Center',
'KA @ The Crossings MS School' :'KA @ The Crossings', 
'KIPP Nashville College Prep School' : 'KIPP Nashville College Prep',
'KIPP Nashville College Prep Elementary School':'KIPP Nashville College Prep Elementary',
'W. A. Bass Learning Center - ALC School' : 'W. A. Bass Learning Center - ALC',
'Carlisle Beasley, Jr Building (Transportation) School' : 'Carlisle Beasley, Jr Building (Transportation)',
'MNPS Supply Center School' : 'MNPS Supply Center',
'MNPS Spencer Lane Warehouse School' : 'MNPS Spencer Lane Warehouse',
'McGruder Family Resource Center School':'McGruder Family Resource Center', 
'MNPS Maint/Supply - GENESCO School' : 'MNPS Maint/Supply - GENESCO',
'Nashville Collegiate Prep School' : 'Nashville Collegiate Prep' ,
'Robertson Academy Gifted School': 'Robertson Academy',
 'Smithson Craighead Middle School':'Smithson Craighead Academy'                                                        
})

In [41]:
Demographics['School_Name'].unique()

array(['A. Z. Kelley Elementary School', 'Alex Green Elementary School',
       'Amqui Elementary School', 'Andrew Jackson Elementary School',
       'Antioch High School', 'Antioch Middle School',
       'Apollo Middle School', 'Aventura Community School',
       'Bellvue Middle School', 'Bellshire Elementary School',
       'Brick Church: A LEAD Public School',
       'Cambridge Early Learning Center', 'Cameron College Preparatory',
       'Cane Ridge Elementary School', 'Cane Ridge High School',
       'Carter-Lawrence Elementary School',
       'Casa Azafran Early Learning Center', 'Chadwell Elementary School',
       'Charlotte Park Elementary School', 'Cockrill Elementary School',
       'Cole Elementary School', 'Cora Howe School',
       'Crieve Hall Elementary School', 'Croft Middle School',
       'Cumberland Elementary School', 'Dan Mills Elementary School',
       'Dodson Elementary School', 'Donelson Middle School',
       'Dupont Elementary School', 'Dupont-Hadley Middle 

In [42]:
Demographics['Asian']  = Demographics['Asian'].fillna(0) 
Demographics['American Indian or Alaska Native']  = Demographics['American Indian or Alaska Native'].fillna(0)
Demographics['Native Hawaiian or Other Pacific Islander'] = Demographics['Native Hawaiian or Other Pacific Islander'].fillna(0)

In [43]:
Demographics['White'] = Demographics['White'].astype(int)
Demographics['Hispanic/Latino'] = Demographics['Hispanic/Latino'].astype(int)
Demographics['Black or African American'] = Demographics['Black or African American'].astype(int)
Demographics['Asian'] = Demographics['Asian'].astype(int)
Demographics['American Indian or Alaska Native'] = Demographics['American Indian or Alaska Native'].astype(int)
Demographics['Native Hawaiian or Other Pacific Islander'] = Demographics['Native Hawaiian or Other Pacific Islander'].astype(int)

In [44]:
Demographics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23860 entries, 0 to 127
Data columns (total 10 columns):
 #   Column                                     Non-Null Count  Dtype 
---  ------                                     --------------  ----- 
 0   School_Name                                23860 non-null  object
 1   Grade                                      23855 non-null  object
 2   Gender                                     23860 non-null  object
 3   Year                                       23860 non-null  object
 4   White                                      23860 non-null  int64 
 5   Hispanic/Latino                            23860 non-null  int64 
 6   Black or African American                  23860 non-null  int64 
 7   Asian                                      23860 non-null  int64 
 8   American Indian or Alaska Native           23860 non-null  int64 
 9   Native Hawaiian or Other Pacific Islander  23860 non-null  int64 
dtypes: int64(6), object(4)
memory usage:

In [45]:
Demographics.to_csv('school_demographics.csv', index = True)

### Part 2
Using the American Community Survey API, obtained for each census tract, the population (B01001_001E in the detailed tables). Hint: Tennessee's FIPS code is 47 and Davidson County's FIPS code is 37.


In [ ]:
endpoint = 'https://api.census.gov/data/2021/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47&in=county:037&key=3a2e429d237f872aa3e2e39bc038ee65608e6090'
Pop_response2021 = requests.get(endpoint)
Population2021 = Pop_response2021.json()
Population2021_df =pd.DataFrame.from_records(Population2021)
Population2021_df

In [ ]:
Population2021_df = Population2021_df.rename(columns = Population2021_df.iloc[0])
Population2021_df = Population2021_df.drop(0)
Population2021_df = Population2021_df.rename(columns = { "B01001_001E" : "2021"})
Population2021_df = Population2021_df.drop(columns = ['state', 'county'])
Population2021_df 

In [ ]:
endpoint = 'https://api.census.gov/data/2020/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47&in=county:037&key=3a2e429d237f872aa3e2e39bc038ee65608e6090'
Pop_response2020 = requests.get(endpoint)
Population2020 = Pop_response2020.json()
Population2020_df =pd.DataFrame.from_records(Population2020)
Population2020_df

In [ ]:
Population2020_df = Population2020_df.rename(columns = Population2020_df.iloc[0])
Population2020_df = Population2020_df.drop(0)
Population2020_df = Population2020_df.rename(columns = { "B01001_001E" : "2020"})
Population2020_df= Population2020_df.drop(columns = ['state', 'county'])
Population2020_df 

In [ ]:
endpoint = 'https://api.census.gov/data/2019/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47&in=county:037&key=3a2e429d237f872aa3e2e39bc038ee65608e6090'
Pop_response2019 = requests.get(endpoint)
Population2019 = Pop_response2019.json()
Population2019_df =pd.DataFrame.from_records(Population2019)
Population2019_df

In [ ]:
Population2019_df = Population2019_df.rename(columns = Population2019_df.iloc[0])
Population2019_df = Population2019_df.drop(0)
Population2019_df = Population2019_df.rename(columns = { "B01001_001E" : "2019"})
Population2019_df = Population2019_df.drop(columns = ['state', 'county'])
Population2019_df 

In [ ]:
endpoint = 'https://api.census.gov/data/2018/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47&in=county:037&key=3a2e429d237f872aa3e2e39bc038ee65608e6090'
Pop_response2018 = requests.get(endpoint)
Population2018 = Pop_response2018.json()
Population2018_df = pd.DataFrame.from_records(Population2018)
Population2018_df

In [ ]:
Population2018_df = Population2018_df.rename(columns = Population2018_df.iloc[0])
Population2018_df = Population2018_df.drop(0)
Population2018_df = Population2018_df.rename(columns = { "B01001_001E" : "2018"})
Population2018_df = Population2018_df.drop(columns = ['state', 'county'])
Population2018_df 

In [ ]:
endpoint = 'https://api.census.gov/data/2017/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47&in=county:037&key=3a2e429d237f872aa3e2e39bc038ee65608e6090'
Pop_response2017 = requests.get(endpoint)
Population2017 = Pop_response2017.json()
Population2017_df = pd.DataFrame.from_records(Population2017)
Population2017_df

In [ ]:
Population2017_df = Population2017_df.rename(columns = Population2017_df.iloc[0])
Population2017_df = Population2017_df.drop(0)
Population2017_df = Population2017_df.rename(columns = { "B01001_001E" : "2017"})
Population2017_df = Population2017_df.drop(columns = ['state', 'county'])
Population2017_df 

In [ ]:
endpoint = 'https://api.census.gov/data/2016/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47&in=county:037&key=3a2e429d237f872aa3e2e39bc038ee65608e6090'
Pop_response2016 = requests.get(endpoint)
Population2016 = Pop_response2016.json()
Population2016_df = pd.DataFrame.from_records(Population2016)
Population2016_df

In [ ]:
Population2016_df = Population2016_df.rename(columns = Population2016_df.iloc[0])
Population2016_df = Population2016_df.drop(0)
Population2016_df = Population2016_df.rename(columns = { "B01001_001E" : "2016"})
Population2016_df = Population2016_df.drop(columns = ['state', 'county'])
Population2016_df 

In [ ]:
endpoint = 'https://api.census.gov/data/2015/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47&in=county:037&key=3a2e429d237f872aa3e2e39bc038ee65608e6090'
Pop_response2015 = requests.get(endpoint)
Population2015 = Pop_response2015.json()
Population2015_df = pd.DataFrame.from_records(Population2015)
Population2015_df

In [ ]:
Population2015_df = Population2015_df.rename(columns = Population2015_df.iloc[0])
Population2015_df = Population2015_df.drop(0)
Population2015_df = Population2015_df.rename(columns = { "B01001_001E" : "2015"})
Population2015_df = Population2015_df.drop(columns = ['state', 'county'])
Population2015_df 

In [ ]:
endpoint = 'https://api.census.gov/data/2014/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47&in=county:037&key=3a2e429d237f872aa3e2e39bc038ee65608e6090'
Pop_response2014 = requests.get(endpoint)
Population2014 = Pop_response2014.json()
Population2014_df = pd.DataFrame.from_records(Population2014)
Population2014_df

In [ ]:
Population2014_df = Population2014_df.rename(columns = Population2014_df.iloc[0])
Population2014_df = Population2014_df.drop(0)
Population2014_df = Population2014_df.rename(columns = { "B01001_001E" : "2014"})
Population2014_df = Population2014_df.drop(columns = ['state', 'county'])
Population2014_df 

In [ ]:
endpoint = 'https://api.census.gov/data/2013/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47&in=county:037&key=3a2e429d237f872aa3e2e39bc038ee65608e6090'
Pop_response2013 = requests.get(endpoint)
Population2013 = Pop_response2013.json()
Population2013_df = pd.DataFrame.from_records(Population2013)
Population2013_df

In [ ]:
Population2013_df = Population2013_df.rename(columns = Population2013_df.iloc[0])
Population2013_df = Population2013_df.drop(0)
Population2013_df = Population2013_df.rename(columns = { "B01001_001E" : "2013"})
Population2013_df = Population2013_df.drop(columns = ['state', 'county'])
Population2013_df 

In [ ]:
endpoint = 'https://api.census.gov/data/2012/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47&in=county:037&key=3a2e429d237f872aa3e2e39bc038ee65608e6090'
Pop_response2012 = requests.get(endpoint)
Population2012 = Pop_response2012.json()
Population2012_df = pd.DataFrame.from_records(Population2012)
Population2012_df

In [ ]:
Population2012_df = Population2012_df.rename(columns = Population2012_df.iloc[0])
Population2012_df = Population2012_df.drop(0)
Population2012_df = Population2012_df.rename(columns = { "B01001_001E" : "2012"})
Population2012_df = Population2012_df.drop(columns = ['state', 'county'])
Population2012_df 

In [ ]:
endpoint = 'https://api.census.gov/data/2011/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47&in=county:037&key=3a2e429d237f872aa3e2e39bc038ee65608e6090'
Pop_response2011 = requests.get(endpoint)
Population2011 = Pop_response2011.json()
Population2011_df = pd.DataFrame.from_records(Population2011)
Population2011_df

In [ ]:
Population2011_df = Population2011_df.rename(columns = Population2011_df.iloc[0])
Population2011_df = Population2011_df.drop(0)
Population2011_df = Population2011_df.rename(columns = { "B01001_001E" : "2011"})
Population2011_df = Population2011_df.drop(columns = ['state', 'county'])
Population2011_df 

In [ ]:
endpoint= 'https://api.census.gov/data/2010/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47&in=county:037&key=3a2e429d237f872aa3e2e39bc038ee65608e6090'
Pop_response2010 = requests.get(endpoint)
Population2010 = Pop_response2010.json()
Population2010_df = pd.DataFrame.from_records(Population2010)
Population2010_df

In [ ]:
Population2010_df = Population2010_df.rename(columns = Population2010_df.iloc[0])
Population2010_df = Population2010_df.drop(0)
Population2010_df = Population2010_df.rename(columns = { "B01001_001E" : "2010"})
Population2010_df = Population2010_df.drop(columns = ['state', 'county'])
Population2010_df 

In [ ]:
endpoint= 'https://api.census.gov/data/2009/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47&in=county:037&key=3a2e429d237f872aa3e2e39bc038ee65608e6090'
Pop_response2009 = requests.get(endpoint)
Population2009= Pop_response2009.json()
Population2009_df=pd.DataFrame.from_records(Population2009)
Population2009_df

In [ ]:
Population2009_df = Population2009_df.rename(columns = Population2009_df.iloc[0])
Population2009_df = Population2009_df.drop(0)
Population2009_df = Population2009_df.rename(columns = { "B01001_001E" : "2009"})
Population2009_df = Population2009_df.drop(columns = ['state', 'county'])
Population2009_df 

In [ ]:
Population = pd.merge(left = Population2021_df, 
         right = Population2020_df, 
         left_on = ['NAME', 'tract'], 
         right_on = ['NAME', 'tract'])
Population

In [ ]:
Population = pd.merge(left = Population, 
         right = Population2019_df, 
         left_on = ['NAME', 'tract'], 
         right_on = ['NAME', 'tract'])
Population

In [ ]:
Population = pd.merge(left = Population, 
         right = Population2018_df, 
         left_on = ['NAME', 'tract'], 
         right_on = ['NAME', 'tract'])
Population

In [ ]:
Population = pd.merge(left = Population, 
         right = Population2017_df, 
         left_on = ['NAME', 'tract'], 
         right_on = ['NAME', 'tract'])
Population

In [ ]:
Population = pd.merge(left = Population, 
         right = Population2016_df, 
         left_on = ['NAME', 'tract'], 
         right_on = ['NAME', 'tract'])
Population

In [ ]:
Population = pd.merge(left = Population, 
         right = Population2015_df, 
         left_on = ['NAME', 'tract'], 
         right_on = ['NAME', 'tract'])
Population

In [ ]:
Population = pd.merge(left = Population, 
         right = Population2014_df, 
         left_on = ['NAME', 'tract'], 
         right_on = ['NAME', 'tract'])
Population

In [ ]:
Population = pd.merge(left = Population, 
         right = Population2013_df, 
         left_on = ['NAME', 'tract'], 
         right_on = ['NAME', 'tract'])
Population

In [ ]:
Population = pd.merge(left = Population, 
         right = Population2012_df, 
         left_on = ['NAME', 'tract'], 
         right_on = ['NAME', 'tract'])
Population

In [ ]:
Population = pd.merge(left = Population, 
         right = Population2011_df, 
         left_on = ['NAME', 'tract'], 
         right_on = ['NAME', 'tract'])
Population

In [ ]:
Population = pd.merge(left = Population, 
         right = Population2010_df, 
         left_on = ['NAME', 'tract'], 
         right_on = ['NAME', 'tract'])
Population

In [ ]:
Population.to_csv('population_census_tract.csv', index = True)

In [ ]:
import re